In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import openpyxl
import statistics
import math
from matplotlib.backends.backend_pdf import PdfPages


def diff_x_i(sig, x_i) :
    return [sig[i]-x_i for i in range(len(sig))]


def diff_a_iw_abs(a_i, a_iw) :
    return abs(a_i - a_iw)



    
    
    
#ファイルの読み込み                                                                                                                         
#file_name1 = sys.argv[1]                                                                                                                   
#file_name2 = sys.argv[2]                                                                                                                   
#file1 = pd.ExcelFile(file_name1+'.xlsx')                                                                                                   
#file2 = pd.ExcelFile(file_name2+'.xlsx')                                                                                                   

def write_data(wb_sheet, sig, column_point, start_row, value_title) :
    wb_sheet.cell(column=column_point, row=1, value=value_title)
    for i, sig_data in enumerate(sig) :
        #print(sig_data)
        wb_sheet.cell(column=column_point, row=i+start_row, value=sig_data)



In [25]:
def inspect(file1, k):
    #ファイル1のデータカウント                                                                                                              
    pattern_dict1 = {}
    sumpsth = 0
    peak = -0.08
    for i, name in enumerate(sheet_names1):
#        print(wb[name]['volt'])
        sheet_df1[i] = file1.parse(name)
        #print(sheet_df1[i])        
        try :
#            for row in wb[name].columns :
#                sig_ori = []
#                for cell in row : 
                    #print(cell.value)
#                    sig_ori.append(cell.value)
            sig_ori = (sheet_df1[i]["volt"]).values
            #sig_ori = sig_ori[150:300]
        except KeyError :
            print("not max data number")
            break
        sig_smooth = [(sig_ori[i]+sig_ori[i+1]+sig_ori[i+2])/3  for i in range(len(sig_ori)-2)]
#        write_data(file1[name], sig_smooth, 2, 3, "smooth")
            
        sig_fire = [ 1 if(sig_smooth[i-3] > sig_smooth[i-2]) & (sig_smooth[i-2] > sig_smooth[i-1]) & (sig_smooth[i-1] > sig_smooth[i]) \
                   & (sig_smooth[i] < sig_smooth[i+1]) & (sig_smooth[i+1] < sig_smooth[i+2]) & (sig_smooth[i+2] < sig_smooth[i+3]) & (sig_ori[i+1] <= peak) \
                    else 0 for i in range(3, len(sig_smooth)-3)]
        
        sig_fire_a = [ 1 if(sig_smooth[i] - sig_smooth[i-1] <= 0) & (sig_smooth[i+1] - sig_smooth[i] >= 0) & (sig_ori[i+1] <= peak) \
                     else 0 for i in range(3, len(sig_smooth) -3)]
        
        
        
        plot_len = len(sig_ori) - (2*k)
        plt.figure(figsize=(10, 4))

        plt.subplot(3, 1, 1)
        plt.subplots_adjust(wspace=0.0, hspace=0)
        #plt.figure()
        start_point = int((len(sig_ori)-plot_len)/2)
        if(start_point == 0) :
            plt.plot(sig_ori)
        else :
            plt.plot(sig_ori[start_point:-start_point])
        #plt.plot(peak_line)     
        #"""       
        for j in range(10) :
            h = 1 + (j/10)
            S_1 = [(max(diff_x_i(sig_ori[i-k:i], sig_ori[i])) + max(diff_x_i(sig_ori[i+1:i+k+1], sig_ori[i])))/2 for i in range(k, len(sig_ori)-k)]
            sig_mean = sum(sig_ori)/len(sig_ori) # m'
            sig_sd = statistics.pstdev(sig_ori) # s'
            S_1_fire = [1 if(S_1[i]-sig_mean > h*sig_sd) & (sig_ori[i+k] < sig_ori[i+k-1]) & (sig_ori[i+k] < sig_ori[i+k+1]) else 0 for i in range(len(S_1))]

            plt.subplot(21, 1, j+8)
            plt.ylim(0.95, 1.05)
            plt.yticks(color="None")
            if(j != 9) :
                plt.xticks(color="None")
            #plt.scatter([i/25 for i in range(len(sig1))],sig1,marker='s', s=10)
            start_point = int((len(S_1_fire)-plot_len)/2)
            if(start_point == 0) :
                plt.scatter([i/25 for i in range(len(S_1_fire))], S_1_fire,marker='s', s=3)
            else :
                plt.scatter([i/25 for i in range(len(S_1_fire[start_point:-start_point]))], S_1_fire[start_point:-start_point],marker='s', s=3)
        #"""           
        pdf.savefig()
        plt.close()  
        #plt.scatter([i/25 for i in range(len(S_1_fire))], S_1_fire,marker='s', s=10)
        #print(S_1_fire)
        

    


In [26]:
#file1 = pd.ExcelFile(sys.argv[1])
#file1 = pd.ExcelFile("/home/nodoka/win/ubuntu/01-data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file1 = pd.ExcelFile("C14-female-L2.xlsx")
file1 = pd.ExcelFile("/home/nodoka/win2/data4.0/all-newtype-bp-ripple/object-25000/B32-HL/B32-object-L3.xlsx")
#file1 = openpyxl.load_workbook("C14-female-L3.xlsx")
#file1 = pd.ExcelFile("/st9/b009vb/01data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file2 = pd.ExcelFile(sys.argv[2])

sheet_df1 = file1.parse(file1.sheet_names, header=None)
#sheet_df2 = file2.parse(file2.sheet_names, header=None)

#cmap = plt.get_cmap("tab20")                                                                                                               

sheet_names1 = file1.sheet_names
#sheet_names2 = file2.sheet_names


k = 5
pdf = PdfPages('parameta-k-h/object/parameta_1h_2k.pdf')
#parameter1_start = int(sys.argv[3])
#parameter1_end = int(sys.argv[4])
parameter1_start = 1
parameter1_end = 1

#parameter2_start = int(sys.argv[5])
#parameter2_end = int(sys.argv[6])
parameter2_start = 1
parameter2_end = 1

#step = int(sys.argv[7])


#file_kull = open("divergence.txt", "a")
#file_data = open("data-ab.txt", "a")
count_data = open("count_data.csv", "a")
inspect(file1, k)
#    wb.save("smooth_data.xlsx")
pdf.close()